In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os 
import skimage

This notebook is only necessary if you need to regenerate the dataset from scratch.

Point the following path to your ImageNet validation directory

In [ ]:
imagenet_validation_dir = "D:\\Datasets\\ILSVRC2012\\Validation\\ILSVRC2012_img_val"

Get all the filenames...

In [ ]:
all_images = os.listdir(imagenet_validation_dir)
all_images = [x for x in all_images if (x[-5:].lower() == ".jpeg") or (x[-4:].lower() == ".jpg")]
print(len(all_images))

Grab the shapes of the images...

In [ ]:
def get_image_shapes():
    shapes = []
    for n, filename in enumerate(all_images):
        if (n % 1000 == 0):
            print(n, end=" ")
        path = os.path.join(imagenet_validation_dir, filename)
        shapes.append(plt.imread(path).shape)
    return shapes

all_image_shapes = get_image_shapes()

Load the ground truth labels of the images

In [ ]:
ground_filename = "ILSVRC2012_validation_ground_truth.txt"
ground_file = open(ground_filename, "r")
ground_lines = ground_file.readlines() 

Create a numpy array. Start the labels from 0 instead of 1.

In [ ]:
labels = np.array([int(x) for x in ground_lines]) - 1

Put the x and y dimensions in an array

In [ ]:
dims = np.array([[x[0],x[1]] for x in all_image_shapes])

Calculate the ratios of width to height

In [ ]:
ratios = np.array([dims[:,0] / dims[:,1], dims[:,1] / dims[:,0]])
ratios = np.max(ratios,axis=0)
ratios

We prefer close as possible to square aspect ratios, so sort the ratios from lowest to highest

In [ ]:
indices = np.argsort(ratios)

Sort dataset based on this critereon.

In [ ]:
sorted_labels = labels[indices]
sorted_ratios = ratios[indices]
sorted_dims = dims[indices]
sorted_images = [all_images[i] for i in indices]

Specify how many items to keep from set per class

In [ ]:
num_per_class = 5

keepers = []
for k in range(1000):
    keepers.append([])

Go through the list of images in sorted order (by aspect ratio). Keep adding to each class until you've reached num_per_class

In [ ]:
for k in range(len(sorted_images)):
    label = sorted_labels[k]
    if (len(keepers[label]) < num_per_class):
        keepers[label].append(k)

So, supremely annoying, there is a difference between the class ordering of VGG-16 under pytorch and the actual ImageNet dataset O_o. So, we need to convert to class indices from the one to the other. The following loads ImageNet 2012's own metadata

In [ ]:
import scipy.io
meta = scipy.io.loadmat('meta.mat')

Make structures for validation set index lookup.

In [ ]:
validation_index_dict = {}
validation_name_dict = {}
validation_names = []

for k in range(1000):
    name = meta["synsets"][k][0][2][0]
    validation_index_dict[name] = k
    validation_name_dict[k] = name
    validation_names.append(name)

In [ ]:
Load the PyTorch VGG16 side indices

In [ ]:
class_id_file = open("imagenet1000_clsid_to_human.txt", "r")
class_dict = eval(class_id_file.read())
class_id_file.close()


Build a conversion dictionary from imagenet 2012 to pytorch VGG16 indices.

In [ ]:
inverse_class_dict = {}
for k in range(1000):
    inverse_class_dict[class_dict[k]] = k

In [ ]:
index_conversion_dict = {}
for k in range(1000):
    index_conversion_dict[k] = inverse_class_dict[validation_names[k]]
    


Make the dataset class directories

In [ ]:
for k in range(1000):
    os.mkdir("dataset/%d" % k)

Resize each of the selected images and write into the dataset directories.

In [ ]:
import skimage.transform
for k in range(1000):
    dirnum = index_conversion_dict[k]
    print("%d " % k, end = " ")
    for sorted_index in keepers[k]:
        image_file = sorted_images[sorted_index]
        path = os.path.join(imagenet_validation_dir, image_file)
        img = plt.imread(path)
        target_path = os.path.join("dataset/%d" % dirnum, image_file)
        #print(target_path)
        
        plt.imsave(target_path, skimage.transform.resize(img, (224, 224, 3)))